# Steady-state BVP from heat conduction

Consider solving
$$
 \begin{cases}-\kappa u_{xx}(x) = g(x),\\
 u(0) = 0,\\
 u(1) = 0.\end{cases}
$$

In [ ]:
using LinearAlgebra, Plots, LaTeXStrings

In [ ]:
m = 49; # number of interior grid points
h = 1/(m+1); # grid spacing
κ = 1.0;
xs = h:h:1-h

In [ ]:
A = SymTridiagonal(fill(2.0,m),fill(-1.0,m-1));
L = A/h^2;

A solution to compare against.  Consider $u(x) = \sin(k \pi x) \exp(-x^2)$.  Then it follows that

\begin{align*}
    -u_{xx}(x) = g(x), \quad g(x) = \left[ k^2 \pi^2 \sin(k \pi x) + 4 x k \pi \cos(k \pi x) + ( 2 - 4x^2 ) \sin(k \pi x) \right] \exp(-x^2).
\end{align*}

In [ ]:
k = 1
g = x -> exp(-x^2)*(k^2*pi^2*sin(k*pi*x) + 4*x*k*pi*cos(k*pi*x) + (2 - 4*x^2)*sin(k*pi*x))
u = x -> sin(k*pi*x)*exp(-x^2)

In [ ]:
b = map(g,xs)
U = L\b
utrue = map(u,xs);

In [ ]:
maximum(abs.(U-utrue))

## Setup function to test convergence

In [ ]:
function solve_DE(m)
    h = 1/(m+1); # grid spacing
    κ = 1.0;
    xs = h:h:1-h
    A = SymTridiagonal(fill(2.0,m),fill(-1.0,m-1));
    L = A/h^2;
    b = map(g,xs)
    U = L\b
    utrue = map(u,xs)
    maximum(abs.(U-utrue))
end

In [ ]:
m = 10;
errs = [solve_DE(m*2^j-1) for j = 0:1:8]
hs = [1/(m*2^j) for j = 0:1:8]
plot(hs,errs,yaxis=:log,xaxis=:log,label="Errors",markershape=:diamond,legend =:bottomright)
plot!(hs,2hs.^2,label=L"y = 2h^2",xlabel = L"h",ylabel = L"\infty\mathrm{-Error}",markershape=:diamond)
    

## Smoothness of solutions

Consider $u(x) = x(x-1) |x-1/2|^2 \mathrm{sgn}(x-1/2)$.  Then

\begin{align*}
    -u_{xx}(x) = g(x), \quad g(x) = 
    \begin{cases}
        \frac{5}{2} - 12 x + 12 x^2 & x > 1/2,\\
        -\frac{5}{2} + 12 x - 12 x^2 & x \leq 1/2.
    \end{cases}
\end{align*}

In [ ]:
k = 1
p = x -> 5/2 - 12*x + 12*x^2
g = x -> x > 1/2 ? -p(x) : p(x)
u = x -> x*(x-1)*abs(x-0.5)^2*sign(x-0.5)

In [ ]:
function solve_DE(m)
    h = 1/(m+1); # grid spacing
    κ = 1.0;
    xs = h:h:1-h
    A = SymTridiagonal(fill(2.0,m),fill(-1.0,m-1));
    L = A/h^2;
    b = map(g,xs)
    U = L\b
    utrue = map(u,xs)
    maximum(abs.(U-utrue))
end

In [ ]:
m = 10;
errs = [solve_DE(m*2^j-1) for j = 0:1:8]
hs = [1/(m*2^j) for j = 0:1:8]
plot(hs,errs,yaxis=:log,xaxis=:log,label="Errors",markershape=:diamond,legend =:bottomright)
plot!(hs,hs.^2,label=L"y = h^2",xlabel = L"h",ylabel = L"\infty\mathrm{-Error}",markershape=:diamond)
    

# Higher-order methods

In [ ]:
using FiniteDifferences, SparseArrays

In [ ]:
method = FiniteDifferenceMethod([-2,-1, 0, 1,2], 2)
display(method)
interior = method.coefs |> Vector;

In [ ]:
method = FiniteDifferenceMethod([-1,0,1,2,3], 2)
display(method)
left = method.coefs |> Vector;

In [ ]:
method = FiniteDifferenceMethod([-3,-2,-1,0,1], 2)
display(method)
right = method.coefs |> Vector;

In [ ]:
m = 49; # number of interior grid points
h = 1/(m+1); # grid spacing
κ = 1.0;
xs = h:h:1-h

In [ ]:
L = spzeros(m,m);
L[1,1:4] = left[2:end]
L[2,1:4] = interior[2:end]
L[end,end-3:end] = right[1:end-1]
L[end-1,end-3:end] = interior[1:end-1]
for i = 3:m-2
    L[i,i-2:i+2] = interior
end
L *= -κ/h^2

In [ ]:
k = 1
g = x -> exp(-x^2)*(k^2*pi^2*sin(k*pi*x) + 4*x*k*pi*cos(k*pi*x) + (2 - 4*x^2)*sin(k*pi*x))
u = x -> sin(k*pi*x)*exp(-x^2)

In [ ]:
b = map(g,xs)
U = L\b
utrue = map(u,xs);

In [ ]:
U - utrue

In [ ]:
function solve_DE_HO(m)
    h = 1/(m+1); # grid spacing
    κ = 1.0;
    xs = h:h:1-h
    
    L = spzeros(m,m);
    L[1,1:4] = left[2:end]
    L[2,1:4] = interior[2:end]
    L[end,end-3:end] = right[1:end-1]
    L[end-1,end-3:end] = interior[1:end-1]
    for i = 3:m-2
        L[i,i-2:i+2] = interior
    end
    L *= -κ/h^2
    b = map(g,xs)
    U = L\b
    utrue = map(u,xs)
    maximum(abs.(U-utrue))
end

In [ ]:
m = 10;
errs = [solve_DE_HO(m*2^j-1) for j = 0:1:8]
hs = [1/(m*2^j) for j = 0:1:8]
plot(hs,errs,yaxis=:log,xaxis=:log,label="Errors",markershape=:diamond,legend =:bottomright)
plot!(hs,hs.^5,label=L"y = h^5",xlabel = L"h",ylabel = L"\infty\mathrm{-Error}",markershape=:diamond)